# Quaternion拡大における素数の偏りの計算

青木美穂さんとの共同研究のためのデモノートブック

このノートブックでは、Quaternion拡大における素数の偏り計算プログラムの使用方法を示します。

## 1. セットアップ

まず、必要なライブラリをインポートします。

In [ ]:
import sys
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display
from collections import Counter

# srcディレクトリをパスに追加
sys.path.append('./src')

print("セットアップ完了")

## 2. システム情報の確認

SageMathとシステムの情報を確認します。

In [ ]:
# SageMathの確認
!sage --version

# システム情報
!./run_scripts.sh info

## 3. テスト実行

小規模なテスト（10^5素数まで）を実行します。

In [ ]:
# Case 1のみをテスト実行
!sage src/main_runner.py --case 1 --all --max-prime 100000 --processes 2

## 4. 結果の確認

計算結果のJSONファイルを読み込んで内容を確認します。

In [ ]:
# データファイルを読み込み
try:
    with open('frobenius_data/case_01_frobenius.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"Case {data['case_id']} の結果:")
    print(f"多項式: {data['polynomial']}")
    print(f"判別式: {data['discriminant']}")
    print(f"m_ρ₀: {data['m_rho0']}")
    print(f"計算した素数の数: {len(data['frobenius_elements'])}")
    
    # フロベニウス元の分布を計算
    frobenius_count = Counter(data['frobenius_elements'].values())
    
    print("\nフロベニウス元の分布:")
    group_names = ['1', '-1', 'i', 'j', 'k', '-i', '-j', '-k']
    total_primes = len(data['frobenius_elements'])
    
    for i in range(8):
        count = frobenius_count.get(i, 0)
        percentage = (count / total_primes) * 100 if total_primes > 0 else 0
        print(f"  g{i} ({group_names[i]}): {count} primes ({percentage:.2f}%)")
        
except FileNotFoundError:
    print("データファイルが見つかりません。まず計算を実行してください。")

## 5. グラフの表示

生成された偏りのグラフを表示します。

In [ ]:
# 統合グラフの表示
try:
    print("Case 1の統合グラフ:")
    display(Image('graphs/case_01_bias_graphs.png'))
except FileNotFoundError:
    print("グラフファイルが見つかりません。")

In [ ]:
# 個別グラフの表示
individual_graphs = [
    ('graphs/case_01_S1_1.png', 'σ = 1'),
    ('graphs/case_01_S2_minus1.png', 'σ = -1'),
    ('graphs/case_01_S3_i.png', 'σ = i'),
    ('graphs/case_01_S4_j.png', 'σ = j'),
    ('graphs/case_01_S5_k.png', 'σ = k')
]

for filename, title in individual_graphs:
    try:
        print(f"\n{title}の偏り:")
        display(Image(filename))
    except FileNotFoundError:
        print(f"{filename} が見つかりません。")

## 6. 複数ケースの実行

複数のケースを順次実行します（時間がかかる場合があります）。

In [ ]:
# Case 1-3のみを実行（テスト用）
for case_id in [1, 2, 3]:
    print(f"\n{'='*50}")
    print(f"Case {case_id} を実行中...")
    print(f"{'='*50}")
    
    !sage src/main_runner.py --case {case_id} --all --max-prime 50000 --processes 2

## 7. 結果の比較

複数ケースの結果を比較します。

In [ ]:
# 複数ケースのデータを読み込んで比較
cases_data = {}

for case_id in [1, 2, 3]:
    filename = f'frobenius_data/case_{case_id:02d}_frobenius.json'
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            cases_data[case_id] = json.load(f)
    except FileNotFoundError:
        print(f"Case {case_id} のデータファイルが見つかりません。")

# 比較表を作成
print("\nケース比較表:")
print("-" * 80)
print(f"{'Case':<6} {'m_ρ₀':<6} {'素数数':<8} {'多項式 (最初の20文字)':<40}")
print("-" * 80)

for case_id, data in cases_data.items():
    m_rho0 = data['m_rho0']
    num_primes = len(data['frobenius_elements'])
    poly_short = data['polynomial'][:40] + "..." if len(data['polynomial']) > 40 else data['polynomial']
    print(f"{case_id:<6} {m_rho0:<6} {num_primes:<8} {poly_short:<40}")

## 8. カスタム分析

データを使って独自の分析を行います。

In [ ]:
# フロベニウス元分布の可視化
if cases_data:
    fig, axes = plt.subplots(1, len(cases_data), figsize=(15, 5))
    if len(cases_data) == 1:
        axes = [axes]
    
    group_names = ['1', '-1', 'i', 'j', 'k', '-i', '-j', '-k']
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray']
    
    for idx, (case_id, data) in enumerate(cases_data.items()):
        # フロベニウス元の分布を計算
        frobenius_count = Counter(data['frobenius_elements'].values())
        total_primes = len(data['frobenius_elements'])
        
        counts = [frobenius_count.get(i, 0) for i in range(8)]
        percentages = [(count / total_primes) * 100 for count in counts]
        
        # 棒グラフを作成
        ax = axes[idx]
        bars = ax.bar(group_names, percentages, color=colors)
        ax.set_title(f'Case {case_id} (m_ρ₀ = {data["m_rho0"]})')
        ax.set_ylabel('Percentage (%)')
        ax.set_xlabel('Frobenius Element')
        
        # 値をバーの上に表示
        for bar, count in zip(bars, counts):
            if count > 0:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                       f'{count}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()
else:
    print("分析するデータがありません。")

## 9. 理論値との比較

実際の計算結果と理論的に期待される分布を比較します。

In [ ]:
# 理論的期待値と実際の分布の比較
if cases_data:
    print("理論値との比較:")
    print("=" * 60)
    
    # Quaternion群Q8の理論的分布（均等分布の場合）
    # g0, g1: 各12.5% (共役類サイズ1)
    # g2-g7: 各12.5% (共役類サイズ2だが、-iは+iと同じ共役類なので実際は25%ずつ)
    theoretical_percentages = {
        0: 12.5,  # g0 (1)
        1: 12.5,  # g1 (-1)
        2: 25.0,  # g2 (i) - includes g5 (-i)
        3: 25.0,  # g3 (j) - includes g6 (-j)
        4: 25.0,  # g4 (k) - includes g7 (-k)
    }
    
    for case_id, data in cases_data.items():
        print(f"\nCase {case_id}:")
        frobenius_count = Counter(data['frobenius_elements'].values())
        total_primes = len(data['frobenius_elements'])
        
        print(f"  総素数数: {total_primes}")
        print(f"  {'Element':<10} {'実際(%)':<10} {'理論(%)':<10} {'差':<10}")
        print(f"  {'-'*40}")
        
        for i in [0, 1, 2, 3, 4]:  # 主要な5つの元のみ
            actual_count = frobenius_count.get(i, 0)
            actual_percentage = (actual_count / total_primes) * 100 if total_primes > 0 else 0
            theoretical = theoretical_percentages.get(i, 0)
            difference = actual_percentage - theoretical
            
            group_names = ['1', '-1', 'i', 'j', 'k']
            print(f"  {group_names[i]:<10} {actual_percentage:<10.2f} {theoretical:<10.2f} {difference:<+10.2f}")

## 10. 全ケース実行（オプション）

時間に余裕がある場合は、全13ケースを実行できます。

**注意**: この処理は非常に時間がかかります（10^6素数で10-15分）。

In [ ]:
# 全ケース実行（コメントアウトしています）
# 実行したい場合は以下のコメントを外してください

# !./run_scripts.sh all

## 11. データのエクスポート

計算結果を他の形式でエクスポートします。

In [ ]:
# CSV形式でデータを出力
import csv

if cases_data:
    with open('frobenius_analysis_results.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['case_id', 'm_rho0', 'total_primes', 'g0_count', 'g1_count', 'g2_count', 'g3_count', 'g4_count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for case_id, data in cases_data.items():
            frobenius_count = Counter(data['frobenius_elements'].values())
            total_primes = len(data['frobenius_elements'])
            
            row = {
                'case_id': case_id,
                'm_rho0': data['m_rho0'],
                'total_primes': total_primes,
                'g0_count': frobenius_count.get(0, 0),
                'g1_count': frobenius_count.get(1, 0),
                'g2_count': frobenius_count.get(2, 0),
                'g3_count': frobenius_count.get(3, 0),
                'g4_count': frobenius_count.get(4, 0)
            }
            writer.writerow(row)
    
    print("分析結果をfrobenius_analysis_results.csvに保存しました。")
else:
    print("エクスポートするデータがありません。")

## 12. まとめ

このノートブックでは以下のことを実行しました：

1. **プログラムのセットアップ**: 必要なライブラリの確認とインポート
2. **テスト実行**: 小規模なケースでの動作確認
3. **結果の分析**: JSONデータの読み込みと内容確認
4. **可視化**: 偏りグラフの表示と分布の可視化
5. **比較分析**: 複数ケースの結果比較
6. **理論値との比較**: 実際の分布と理論的期待値の比較
7. **データエクスポート**: CSV形式での結果出力

### 次のステップ

- より大規模な計算（10^6素数）を実行する
- 全13ケースの詳細な比較分析
- 偏り係数の理論値と実測値の詳細な検証
- 新しいケースの追加と分析

### 参考資料

- 青木美穂, 小山信也. "Chebyshev's Bias against Splitting and Principal Primes in Global Fields", Journal of Number Theory 245, 233-262 (2023).
- S. Omar. "Central values of Artin L-functions for quaternion fields", ANTS-IV (2000).